In [1]:
from sage.all import *
import numpy as np
import scipy as sp

maxDegree = 5 # the largest homological degree (in the dga) that you want to calculate homology in: some incomplete info will be 
# calc'd in higher degree for simplicity. Actually we will calculate everything in word length (=tensor power) at most maxDegree, so this would more
# properly be called maxTensorPower
n = 6 # half of the number of strands of the even Temperley-Lieb algebra you want to work with

# module of generators
index = ellipsis_range(1,Ellipsis,n)
Y = CombinatorialFreeModule(ZZ, index)
y = Y.basis()

# construct the tensor square, which will be the codomain of the differential on the generators
T = tensor([Y, Y])
# define differential Y via def on basis
on_basis = lambda k: T.sum([binomial(k,i) * tensor([Y.monomial(i), Y.monomial(k-i)]) for i in range(1,k)])
d1 = Y.module_morphism(on_basis, codomain=T) # the differential on the generators
# differential1to2(y[3]) # uncomment me to check your basis vectors are going where you think

# from here we can do everything with matrices, without the indexing getting too bad
d1matrix = d1.matrix().sparse_matrix(); # matrix rep of d1
# we now convert everything to numpy/scipy for more efficient (?) sparse manipulations
weightVector1 = np.asarray(index) # ordered list of the weights of the generators

d1sparce = sp.sparse.csr_array(np.array(d1matrix))
kleineId = sp.sparse.identity(n, dtype='int64', format='csr') # format='dia' (a type of sparse array) may improve performance

# now make the matrix describing differential on length k tensors, each k, together with list of weights
diffMatrices = []
diffMatrices.append(d1sparce)
weightVectors = []
weightVectors.append(weightVector1)
tensorPower = 2
print("Applying the Leibniz rule repeatedly:")

while tensorPower <= maxDegree + 1 : # also need inbound diff in top degree
    print(tensorPower)
    # calculate new differential according to Leibniz
    grootId = sp.sparse.identity(n**(tensorPower - 1), dtype='int64', format='csr')
    newDiff = - sp.sparse.kron(kleineId, diffMatrices[tensorPower - 2]) + sp.sparse.kron(d1sparce,grootId) # taking advantage of all generators living in odd degree to simplify
    diffMatrices.append(newDiff)
    print("done diff, doing weights")
    weightVectors.append(np.fromfunction(lambda i, j: weightVectors[tensorPower - 2][j] + weightVector1[i], (n, n**(tensorPower - 1)), dtype='int64').flatten())
    tensorPower += 1

# the weights are for the columns of each matrix. These are equal to those of the rows of the preceding. We will also want to know which rows we can discard from the last one, so we record its row weights too
weightVectors.append(np.fromfunction(lambda i, j: weightVectors[tensorPower - 2][j] + weightVector1[i], (n, n**(tensorPower - 1)), dtype='int64').flatten())  

print("Success!")

if np.log2(n**tensorPower) >= 64:
    print("I made a list that was really big to store weights - about 2 to the:")
    print(np.log2(n**tensorPower))
    print("long. It's so big that the 64 bit integers we used to index it aren't big enough - THERE WILL BE ERRORS!")

#testPower = 7
#dex = 0
#counter = 0
#errors = []
#while dex < len(list(weightVectors[testPower - 1])):
#    if list(weightVectors[testPower - 1])[dex] == testPower:
#        counter += 1
#        errors.append(dex)
#    dex += 1
#print(counter)
#print(errors)
#print(len(weightVectors[testPower - 1]))
#print(list(weightVectors[testPower - 1]))
#print("differentials out of each tensor power are: ")
#print(diffMatrices)

Applying the Leibniz rule repeatedly:
2
done diff, doing weights
3
done diff, doing weights
4
done diff, doing weights
5
done diff, doing weights
6
done diff, doing weights
Success!


In [2]:
# now we split the differential matrices up according to the weights: this will also homogenize degrees
# we will format the answer as:
diffMatricesRefined = [] # will be list of lists: diffMatricesRefined[i][j]=diff out of tensor power i+1, weight j+1
newTensorPower = 1

print("Decomposing each tensor power:")

while newTensorPower <= maxDegree + 1 :
    currentWeight = 1
    currentMatrix = diffMatrices[newTensorPower - 1]
    currentMatrixRefined = [] # we will build this list, then append a copy to diffMatricesRefined
    while currentWeight < newTensorPower :
        currentMatrixRefined.append("no elements of weight less than monomial length")
        currentWeight += 1
    
    # now we hit currentWeight = newTensorPower, and there begins to be stuff to do
    while currentWeight <= n * newTensorPower : # the element of largest weight is y_n^{\otimes newTensorPower}, of weight n*newTensorPower
        colsToKeep = list(weightVectors[newTensorPower - 1] == currentWeight)
        rowsToKeep = list(weightVectors[newTensorPower] == currentWeight)
        colAndRowRestriction = currentMatrix[rowsToKeep, :][:, colsToKeep]
        currentMatrixRefined.append(copy(colAndRowRestriction))
        currentWeight += 1
    #print("At tensor power:")
    #print("Differential decomposes into weight blocks:")
    #print(currentMatrixRefined)
    diffMatricesRefined.append(list(currentMatrixRefined))
    print(newTensorPower)
    newTensorPower += 1

print("Success!")
# diffMatricesRefined

Decomposing each tensor power:
1
2
3
4
5
6
Success!


In [3]:
# Now we calculate the homology of each chain complex
# (for safekeeping) here's an example of how to set up a simple homologically graded cx 
# D = ChainComplex({7: matrix(ZZ, 2, 2, [1,0,0,2])}, degree = -1); D
# ascii_art(D)
from sage.misc.converting_dict import KeyConvertingDict
# we calculated enough tensor powers that we have all outbound differentials for tensor powers up to maxDegree+1
maxTensorPower = maxDegree # hopefully the logic will be clearer

print("Computing some homology for Temperley--Lieb over Z, parameter 0, strands:")
print(2*n)
print("Answer complete thru degree:")
print(maxDegree)
print(" ")

# pari.allocatemem(10**10) # this seems to be how you allocate more memory - when I tried to use it I just seemed to replace one crash with another

complexWeight = 2 # start in weight 2, where first nonzero outbound diff goes
while complexWeight <= n * maxTensorPower: # this is the highest weight on which we recorded a differential
    # it seems like the easiest way to make a homologically graded ch cx is with an integer dictionary
    dictionary = KeyConvertingDict(int)
    homologicalDegree = complexWeight + 1 # smallest degree in which this weight may have nonzero outbound diff
    print("weight:")
    print(complexWeight)
    # print("working through homological degrees:")
    while(homologicalDegree <= 2 * complexWeight - 1): # the weight w complex is supported in degrees (perhaps a proper) subset of the interval [w,2w-1]
        tensorPower = 2 * complexWeight - homologicalDegree
        # recall monomial length is twice weight minus degree, extra -1 for python indexing
        # print(homologicalDegree)
        if tensorPower > 0 and tensorPower <= maxTensorPower and homologicalDegree >= complexWeight + 1 and complexWeight <= n*tensorPower:
            dictionary[homologicalDegree] = Matrix(ZZ,diffMatricesRefined[tensorPower - 1][complexWeight - 1].toarray())
        homologicalDegree += 1
    # print(dictionary)
    C = ChainComplex(dictionary, degree = -1) # form a chain complex from our lists of matrices: we've already set the degrees correctly
    # complexes.append(C)

    # now output the complex
    # print("The chain complex is:")
    # print(ascii_art(C))

    # now output homology: print(homologies) does fine but includes information from above maxDegree, which won't be correct in general, so this
    # next chunk is really just producing the same answer restricted to degrees at most maxDegree
    # print(C.homology())
    readHomologyInDegree = int(complexWeight)
    readHomologyInTensorPower = 2 * complexWeight - readHomologyInDegree
    homologies = KeyConvertingDict(int)
    while(readHomologyInDegree >= complexWeight and complexWeight <= n*readHomologyInTensorPower):
        if(readHomologyInTensorPower <= maxTensorPower):
            homologies[readHomologyInDegree] = C.homology(deg = readHomologyInDegree)
        readHomologyInDegree += 1
        readHomologyInTensorPower = 2 * complexWeight - readHomologyInDegree
    print("Homology indexed by degree:")
    print(homologies)
    print("  ")
    
    
    complexWeight += 1

Computing some homology for Temperley--Lieb over Z, parameter 0, strands:
12
Answer complete thru degree:
5
 
weight:
2
Homology indexed by degree:
{2: C2, 3: 0}
  
weight:
3
Homology indexed by degree:
{3: C2, 4: C3, 5: 0}
  
weight:
4
Homology indexed by degree:
{4: C2, 5: C3, 6: C2, 7: 0}
  
weight:
5
Homology indexed by degree:
{5: C2, 6: 0, 7: C2, 8: C5, 9: 0}
  
weight:
6
Homology indexed by degree:
{7: 0, 8: C6, 9: C10, 10: 0, 11: 0}
  
weight:
7
Homology indexed by degree:
{9: C6, 10: C2, 11: 0, 12: Z}
  
weight:
8
Homology indexed by degree:
{11: C2, 12: C2, 13: Z x C4, 14: 0}
  
weight:
9
Homology indexed by degree:
{13: C2, 14: C2 x C2 x C4, 15: C3, 16: 0}
  
weight:
10
Homology indexed by degree:
{15: C2^5, 16: C3 x C3 x C30, 17: C5, 18: 0}
  
weight:
11
Homology indexed by degree:
{17: C6 x C30, 18: C2 x C10, 19: 0, 20: 0}
  
weight:
12
Homology indexed by degree:
{19: C2 x C2 x C6 x C6, 20: C5 x C30, 21: 0, 22: 0}
  
weight:
13
Homology indexed by degree:
{21: C2 x C2 x C